In [2]:
import pandas as pd
from sqlalchemy import create_engine

server = "DESKTOP-N66DL73\\SQLEXPRESS"
database = "AdventureWorks2022"

# SQLAlchemy engine yaradılır
engine = create_engine(f"mssql+pyodbc://{server}/{database}?trusted_connection=yes&driver=SQL+Server")

try:
    # SQL sorğusu
    query = """
select	
	sh.SalesOrderID,
	sh.SalesPersonID,
	pep.FirstName,
	pep.LastName,
	sd.ProductID,
	sh.OrderDate,
	sh.DueDate,
	sh.ShipDate,
	sd.UnitPrice,
	sh.SubTotal,
	sh.TaxAmt
from 
	sales.SalesOrderHeader as sh 
join
	Sales.SalesOrderDetail as sd
on
	sh.SalesOrderID=sd.SalesOrderID
join
	Production.Product as pp
on 
	sd.ProductID=pp.ProductID
join
	Sales.SalesTerritory as st
on
	sh.TerritoryID=st.TerritoryID
join
	Person.Person pep
on
	pep.BusinessEntityID=sh.SalesPersonID"""  
    # Pandas ilə sorğunu icra edirik
    df = pd.read_sql(query, engine)
    
    #Transaksiyaların addımları
    #1.firstname ve lastname sutunlarını birlsdirərək fullname adlı yeni bir sütun yaradılır
    df['FullName'] = df['FirstName'] + ' ' + df['LastName']
    #2. OrderDate, DueDate və ShipDate sütunlarının tipini datetime tipinə çeviririk
    df['OrderDate'] = pd.to_datetime(df['OrderDate'])
    df['DueDate'] = pd.to_datetime(df['DueDate'])
    df['ShipDate'] = pd.to_datetime(df['ShipDate'])
    #3. OrderDate sütunundan yeni bir sütun yaradılır və bu sütunun adı OrderYear olur
    df['OrderYear'] = df['OrderDate'].dt.year
    #4. firstname ve lastname sutunlarını birlsdirərək fullname adlı yeni bir sütun yaradılır
    df['FullName'] = df['FirstName'] + ' ' + df['LastName']
    #4. OrderDate sütunundan yeni bir sütun yaradılır və bu sütunun adı OrderMonth olur
    df['OrderMonth'] = df['OrderDate'].dt.month
    #5.Subtotalı 0.18 vurularaq ƏDV hesablanır
    df['ƏDV']=df["SubTotal"]*0.18
    #6.Dublicate olan sətirləri silirik
    df_unique = df.drop_duplicates()
    #LOAD HISSESİ
    pg_user = "postgres"
    pg_password = "karimovv1818"
    pg_host = "localhost"   
    pg_port = "5432"
    pg_database = "mydata"

    pg_engine = create_engine(f"postgresql://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_database}")
    #Dataframe postgre sql serverə yüklənir
    df_unique.to_sql('sales', pg_engine, index=False, if_exists='replace')
    print("Data successfully loaded to PostgreSQL")
except Exception as e:
    print("Error:", e)

Data successfully loaded to PostgreSQL
